In [ ]:
import pandas as pd
import time
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException


In [133]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [ ]:
def getCoinInfo(link):
    driver.get(link)
    count = 1
    info, brand, end_date, limit, min, customer = [], [], [], [], [], []
    i = 1

    #Load tất cả các trang
    while True:
        try:
            # Chờ nút "Load More" hiển thị và có thể nhấn
            label = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="list_coupon"]/div/div[5]/label'))
            )

            # Thực hiện click
            label.click()
            print("Clicked on button next page!")

            # Sau khi nhấn, chờ để trang cập nhật và nút tiếp theo xuất hiện
            WebDriverWait(driver, 10).until(
                EC.staleness_of(label)  # Đợi cho phần tử hiện tại trở nên cũ và bị thay thế bởi phần tử mới
            )

        except (TimeoutException, NoSuchElementException):
            # Nếu không tìm thấy nút hoặc hết thời gian chờ, kết thúc vòng lặp
            print("No more Load More button found or timeout reached.")
            break
    
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web

    while True:
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(5)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        if (screen_height) * i > scroll_height:
            break 
            
    
    # info1 =  driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[3]/p')
    # info =  [s.text for s in info1] + info       
    brand1 = driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[2]/a')
    brand = [el.find_element(By.TAG_NAME, 'img').get_attribute("alt") for el in brand1] + brand  
    # print(brand)

    info1 =  driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[3]/p')
    info =  [s.text for s in info1] + info

    end_date1 =  driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[5]/div/div[1]/div[2]')
    end_date =  [s.text for s in end_date1] + end_date

    limit1 =  driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[5]/div/div[2]/div[2]')
    limit =  [s.text for s in limit1] + limit

    min1 =  driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[5]/div/div[3]/div[2]')
    min =  [s.text for s in min1] + min

    customer1 =  driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[5]/div/div[4]/div[2]')
    customer =  [s.text for s in customer1] + customer



    print(end_date)
    
    coinmarketcap1 = pd.DataFrame(list(zip(brand, info, end_date, limit, min, customer)),
                        columns=['brand', 'info', 'end_date', 'limit', 'min', 'customer']) 
    print("Crawl Complete")
    return coinmarketcap1

url = 'https://iprice.vn/coupons/'
df = getCoinInfo(url)
driver.close()

Clicked on button next page!
No more Load More button found or timeout reached.


NameError: name 'screen_height' is not defined

In [127]:
# Lấy file CSV
df_unique = df.drop_duplicates(keep='first')
df_unique.to_csv('D://Exam/Coin.csv', index=False)


In [ ]:
//*[@id="list_coupon"]/div[1]/div[5]/label
//*[@id="list_coupon"]/div[2]/div[5]/div[1]/label
//*[@id="list_coupon"]/div[3]/div[5]/label